### Stored Procedures, TRY/CATCH (Handlers)

In real systems, SQL is executed as part of repeatable jobs:
- monthly reporting
- daily loads
- error handling and logging
- parameter-driven queries

MySQL implements TRY/CATCH-style behavior using **HANDLERS** in stored programs.

### Stored Procedure — Monthly order count report

The reporting team runs the same monthly order count query frequently.
- Create a stored procedure
- Execute it using `CALL`

In [ ]:
DELIMITER $$

DROP PROCEDURE IF EXISTS get_monthly_orders $$

CREATE PROCEDURE get_monthly_orders()
BEGIN
  SELECT
    DATE_FORMAT(order_date, '%Y-%m') AS order_month,
    COUNT(*) AS order_count
  FROM orders
  GROUP BY order_month
  ORDER BY order_month;
END $$

DELIMITER ;

### Running the stored procedure

In [ ]:
CALL get_monthly_orders();

## TRY / CATCH 

During a batch step, inserts might fail (e.g., bad foreign key, duplicates).
We want to:
- capture the error
- return a controlled message (or log it)
- avoid confusing failures

MySQL uses:
- `DECLARE ... HANDLER FOR SQLEXCEPTION`
inside stored procedures or blocks.

In [ ]:
DELIMITER $$

DROP PROCEDURE IF EXISTS demo_error_handler $$

CREATE PROCEDURE demo_error_handler()
BEGIN
  DECLARE had_error TINYINT DEFAULT 0;

  DECLARE CONTINUE HANDLER FOR SQLEXCEPTION
  SET had_error = 1;

  -- Intentional example: this may fail if customer_id does not exist
  INSERT INTO orders(order_date, order_customer_id, order_status)
  VALUES (NOW(), 99999999, 'CREATED');

  IF had_error = 1 THEN
    SELECT 'An error occurred during the insert. The handler captured it.' AS message;
  ELSE
    SELECT 'Insert succeeded.' AS message;
  END IF;
END $$

DELIMITER ;

#### Execute the handler demo

In [ ]:
CALL demo_error_handler();

### Top-N queries — Top 10 products by revenue

Merchandising needs the top performers for promotion planning.
- Aggregate revenue by product
- Sort descending
- Return Top-N using `LIMIT`

In [ ]:
SELECT
  oi.order_item_product_id AS product_id,
  SUM(oi.order_item_subtotal) AS total_revenue
FROM order_items oi
GROUP BY oi.order_item_product_id
ORDER BY total_revenue DESC
LIMIT 10;

### Indexes

A common production query is: “Fetch all orders for a customer quickly”.

An index on `orders(order_customer_id)` allows MySQL to avoid scanning all rows.

Create an index and validate with `EXPLAIN`.

In [ ]:
CREATE INDEX IF NOT EXISTS idx_orders_customer_id
ON orders(order_customer_id);